In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

# Hyperparameters
img_size = 180
batch_size = 32
num_classes = 5
epochs = 50

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Load data
train_generator = train_datagen.flow_from_directory(
    'Images/',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Images/',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Print class weights to check for imbalance
class_weights = {}
total_samples = len(train_generator.classes)
label_counts = np.bincount(train_generator.classes)
for i, count in enumerate(label_counts):
    class_weights[i] = total_samples / (len(label_counts) * count)

print("Class Weights:", class_weights)
print("Class Names:", train_generator.class_indices)

# Use Transfer Learning (MobileNetV2)
base_model = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_size, img_size, 3)
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    class_weight=class_weights
)

# Save the model
model.save('Flower_classification.h5')

# Evaluation
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

Found 6772 images belonging to 5 classes.
Found 1691 images belonging to 5 classes.
Class Weights: {0: 0.9715925394548063, 1: 0.7995277449822904, 2: 1.6259303721488596, 3: 0.951123595505618, 4: 0.9491240364400841}
Class Names: {'daisy': 0, 'dandelion': 1, 'iris': 2, 'rose': 3, 'sunflower': 4}


/var/folders/x1/lkxlqzsn619fk3ylr3hzgb0w0000gn/T/ipykernel_61713/180239757.py:55: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 72s 330ms/step - accuracy: 0.4678 - loss: 1.4734 - val_accuracy: 0.8492 - val_loss: 0.4738
Epoch 2/50
  1/211 ━━━━━━━━━━━━━━━━━━━━ 52s 252ms/step - accuracy: 0.8750 - loss: 0.5051

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


211/211 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - accuracy: 0.8750 - loss: 0.5051 - val_accuracy: 0.8528 - val_loss: 0.4712
Epoch 3/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 60s 286ms/step - accuracy: 0.8051 - loss: 0.5500 - val_accuracy: 0.8834 - val_loss: 0.3489
Epoch 4/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - accuracy: 0.7812 - loss: 0.3914 - val_accuracy: 0.8762 - val_loss: 0.3609
Epoch 5/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 66s 307ms/step - accuracy: 0.8475 - loss: 0.4360 - val_accuracy: 0.8960 - val_loss: 0.3009
Epoch 6/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 16s 77ms/step - accuracy: 0.9062 - loss: 0.2188 - val_accuracy: 0.9020 - val_loss: 0.2851
Epoch 7/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 58s 275ms/step - accuracy: 0.8717 - loss: 0.3566 - val_accuracy: 0.9087 - val_loss: 0.2634
Epoch 8/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.9062 - loss: 0.2982 - val_accuracy: 0.9153 - val_loss: 0.2626
Epoch 9/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 60s 286ms/step - accuracy: 0.8876 - loss: 0.3094 - val_acc

53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 178ms/step - accuracy: 0.9223 - loss: 0.2044
Test accuracy: 0.9284446835517883
